<a href="https://colab.research.google.com/github/alixintong/btt-ai-know-your-customer/blob/jess/bounding_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this cell allows access to our folder

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/gdrive/MyDrive/BTTAI_AmericanExpress1'

Mounted at /content/drive


In [ ]:
# cell for imports - add as needed
import torch 
import cv2
import numpy as np 
import pandas as pd
import os, os.path
import re
from fastai.vision.all import *

In [ ]:
# for reference
#   - fastai segmentation: https://walkwithfastai.com/Segmentation
#   - fastai object detection: https://walkwithfastai.com/Object_Detection

In [ ]:
path = '/content/drive/MyDrive/BTTAI_AmericanExpress1/datasets/sampleDataset/frames'

files = get_image_files(path + "/datasheet001_frames")
len(files)

214